In [1]:
import pandas as pd
from quote_extractor import QuoteExtractor

In [2]:
class DatasetConfig():
    input_path: str
    output_path: str
    quote_annotations_path: str
    sheet_name: str
    id_col: str
    usecols: list[str]

    def __init__(self, input_path: str, output_path: str, quote_annotations_path: str, sheet_name: str, id_col: str, usecols: list[str]):
        self.input_path = input_path
        self.output_path = output_path
        self.quote_annotations_path = quote_annotations_path
        self.sheet_name = sheet_name
        self.id_col = id_col
        self.usecols = usecols

In [3]:
fakespeak_config = DatasetConfig(
    # file_path="/content/drive/My Drive/fake_news_over_time/Fakespeak_ENG_modified.xlsx",
    input_path="./data/Fakespeak-ENG/Fakespeak-ENG modified.xlsx",
    output_path="./data/Fakespeak-ENG/Analysis_output/Fakespeak_quotes.xlsx",
    quote_annotations_path="./data/Fakespeak-ENG/Analysis_output/quote_annotations.json",
    sheet_name="Working",
    id_col="ID",
    usecols=['ID', 'combinedLabel', 'originalTextType', 'originalBodyText', 'originalDateYear'],
)

misinfotext_config = DatasetConfig(
    input_path="./data/MisInfoText/PolitiFact_original_modified.xlsx",
    output_path="./data/MisInfoText/Analysis_output/MisInfoText_quotes.xlsx",
    quote_annotations_path="./data/MisInfoText/Analysis_output/quote_annotations.json",
    sheet_name="Working",
    id_col="factcheckURL",
    usecols=None,
)

In [4]:
using_dataset = misinfotext_config

In [5]:
dataset_df = pd.read_excel(
    using_dataset.input_path, 
    sheet_name=using_dataset.sheet_name, 
    usecols=using_dataset.usecols)
dataset_df

,factcheckURL,originalURL,originalBodyText,originalHeadline,originalTextType,originalDate,originalDateYear
0,http://www.politifact.com/arizona/statements/2...,https://associatedmediacoverage.com/three-stat...,Residents of multiple states will be asked to ...,Multiple States Have Agreed To Implement A ‘Tw...,News and blog,2016-05-06,2016
1,http://www.politifact.com/california/statement...,https://users.focalbeam.com/fs/distribution:wl...,"Sacramento, CA - United States Senator Dianne ...",U.S. Senator Dianne Feinstein Opposes Prop. 64...,Press release,2016-07-12,2016
2,http://www.politifact.com/california/statement...,http://www.sacbee.com/opinion/op-ed/soapbox/ar...,We should anticipate black and gray markets in...,Why you should buy a locking gasoline cap,News and blog,2017-08-04,2017
3,http://www.politifact.com/california/statement...,https://nocagastax.com/california-gas-tax-hike...,As a ballot initiative calling for repeal of a...,California Gas-Tax-Hike Repeal Campaign Heats Up,News and blog,2017-06-15,2017
4,http://www.politifact.com/california/statement...,https://chu.house.gov/media-center/press-relea...,"WASHINGTON, DC The House of Representatives t...","Rep. Chu Decries ""Heartless"" ACA Repeal Vote",Press release,2017-05-04,2017
...,...,...,...,...,...,...,...
650,http://www.politifact.com/wisconsin/statements...,https://x.com/ScottWalker/status/9428776407421...,Road projects across the state are staying on ...,NaN,Social media,2017-12-18,2017
651,http://www.politifact.com/wisconsin/statements...,https://x.com/ScottWalker/status/9511017961011...,The last thing we need is more Madison in our ...,NaN,Social media,2018-01-10,2018
652,http://www.politifact.com/wisconsin/statements...,https://x.com/MahlonMitchell/status/9538161542...,When \n@ScottWalker\n told firefighters we did...,NaN,Social media,2018-01-18,2018
653,http://www.politifact.com/wisconsin/statements...,http://dailycaller.com/2018/01/25/hey-look-sen...,"Now that its 2018, an election year, I would l...",HEY LOOK! Senator Tammy Baldwin Is Back In Wis...,News and blog,2018-01-25,2018


In [6]:
quote_extractor = QuoteExtractor("en_core_web_lg", "./quote_verb_list.txt")

c:\Users\Adam\AppData\Local\Python\pythoncore-3.11-64\Lib\site-packages\coreferee\manager.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [7]:
quote_annotations = quote_extractor.run_multiple(dataset_df[using_dataset.id_col], dataset_df["originalBodyText"])

Preprocessing texts...
Creating spacy docs...
Extracting quotes


In [ ]:
dataset_df["quotes"] = [[doc["quote"] for doc in docs] 
                        for docs in quote_annotations]
dataset_df["quote_lengths"] = [[doc["quote_token_count"] for doc in docs] 
                        for docs in quote_annotations]
dataset_df

In [ ]:
all_quotes_df = dataset_df.explode(["quotes", "quote_lengths"])\
    .rename(columns={"quotes": "quote"})\
    .rename(columns={"quote_lengths": "quote_length"})
all_quotes_df = all_quotes_df[all_quotes_df["quote"].notna()]
all_quotes_df

In [ ]:
grouped_by_year = all_quotes_df.groupby(by="originalDateYear")
years = grouped_by_year.groups
years_dfs = [grouped_by_year.get_group(year) for year in years]
years_dfs[0].head()

In [ ]:
num_quotes_per_year = grouped_by_year["quote"].count()
num_quotes_per_year

In [ ]:
quote_length_summary_df = pd.DataFrame(
    [df["quote_length"].convert_dtypes().describe() for df in years_dfs],
    index=pd.Index(data=years, name="year")
)
quote_length_summary_df

In [ ]:
only_news_blog_social_media_df = all_quotes_df[(all_quotes_df["originalTextType"] == "News and blog") | (all_quotes_df["originalTextType"] == "Social media")]
only_news_blog_social_media_df

In [ ]:
grouped_by_year_news_blog_social_media = only_news_blog_social_media_df.groupby(by="originalDateYear")
years_news_blog_social_media = grouped_by_year_news_blog_social_media.groups
years_news_blog_social_media_dfs = [grouped_by_year_news_blog_social_media.get_group(year) for year in years_news_blog_social_media]
years_news_blog_social_media_dfs[0].head()

In [ ]:
num_quotes_per_year_news_blog_social_media = grouped_by_year_news_blog_social_media["quote"].count()
num_quotes_per_year_news_blog_social_media

In [ ]:
quote_length_summary_news_blog_social_media_df = pd.DataFrame(
    [df["quote_length"].convert_dtypes().describe() for df in years_news_blog_social_media_dfs],
    index=pd.Index(data=years_news_blog_social_media, name="year")
)
quote_length_summary_news_blog_social_media_df

In [ ]:
writer = pd.ExcelWriter(using_dataset.output_path, engine="xlsxwriter")

for df, year in zip(years_dfs, years):
    df\
        .sort_values(by="quote_length")\
        .to_excel(writer, sheet_name=str(year), index=False, columns=["quote", "quote_length", "originalDateYear"])

num_quotes_per_year.to_excel(writer, sheet_name="Number of quotes")
quote_length_summary_df.to_excel(writer, sheet_name="Quote length summary")

writer.close()

In [ ]:
output_path = using_dataset.output_path
output_path_split = output_path.split("/")
output_path_split.insert(len(output_path_split) - 1, "news_blog_and_social_media")
output_path_news_blog_social_media = "/".join(output_path_split)
output_path_news_blog_social_media

In [ ]:
writer = pd.ExcelWriter(output_path_news_blog_social_media, engine="xlsxwriter")

for df, year in zip(years_news_blog_social_media_dfs, years_news_blog_social_media):
    df\
        .sort_values(by="quote_length")\
        .to_excel(writer, sheet_name=str(year), index=False, columns=["quote", "quote_length", "originalDateYear"])

num_quotes_per_year_news_blog_social_media.to_excel(writer, sheet_name="Number of quotes")
quote_length_summary_news_blog_social_media_df.to_excel(writer, sheet_name="Quote length summary")

writer.close()